<a href="https://colab.research.google.com/github/kuuuun/python_jupyter_notes/blob/main/chaptor05_ufuncs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# chaptor05 Ufuncs

In [1]:
import numpy as np

In [2]:
import math
import time

# 准备 500 万个随机数
N = 5_000_000
rng = np.random.default_rng(42)
data_np = rng.random(N)
data_list = data_np.tolist()

print(f"数据量: {N}")

# === 选手 A: Python 列表推导式 + math.sin ===
start = time.time()
res_list = [math.sin(x) for x in data_list]
end = time.time()
print(f"Python Math 耗时: {end - start:.4f} 秒")

# === 选手 B: NumPy Ufunc ===
start = time.time()
# np.sin 就是一个典型的 Ufunc
res_np = np.sin(data_np)
end = time.time()
print(f"NumPy Ufunc 耗时: {end - start:.4f} 秒")

数据量: 5000000
Python Math 耗时: 0.2954 秒
NumPy Ufunc 耗时: 0.0556 秒


## 内存优化：out 参数的妙用

In [5]:
import numpy as np

# 创建一个较大的数组
large_arr = np.arange(10)
print("原始地址:", id(large_arr))

# 方式 1：常规写法 (创建了新数组，然后重新赋值给变量名)
# 这在中间过程中会产生临时数组
large_arr = np.multiply(large_arr, 2)
print("方式1后地址:", id(large_arr)) # 地址变了，说明发生了内存分配

# 方式 2：使用 out 参数 (原地修改)
# 没有任何临时数组产生
np.multiply(large_arr, 2, out=large_arr)
print("方式2后地址:", id(large_arr)) # 地址没变！

# 甚至可以用切片作为 out 的目标
# 只把前 5 个元素变成负数
np.negative(large_arr[:5], out=large_arr[:5])
print("局部修改后:", large_arr)

原始地址: 138461323149104
方式1后地址: 138461323147760
方式2后地址: 138461323147760
局部修改后: [  0  -4  -8 -12 -16  20  24  28  32  36]


In [8]:
arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])
print(arr.shape)

# 等同于 np.sum(arr, axis=0)
result_a0 = np.add.reduce(arr, axis=0)
print(result_a0)
# 输出: [12 15 18]
result_a1 = np.add.reduce(arr, axis=1)
print(result_a1)

(3, 3)
[12 15 18]
[ 6 15 24]


In [13]:
arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])
print(arr.shape)

# 等同于 np.sum(arr, axis=0)
result_a0 = np.sum(arr,axis=0)
print(result_a0)
# 输出: [12 15 18]
result_a1 = np.sum(arr,1)
print(result_a1)

(3, 3)
[12 15 18]
[ 6 15 24]


检查数组是否已排序：
这是一个经典的用法。通过比较“当前元素 < 下一个元素”是否在所有位置都成立。

In [14]:
arr = np.array([1, 3, 2, 4])

# 检查是否 arr[i] < arr[i+1] 对所有 i 都成立
# np.less 就是 <
is_sorted = np.logical_and.reduce(np.less(arr[:-1], arr[1:]))
print(is_sorted)  # False (因为 3 > 2)

False


检查数值范围：
虽然通常用 (a < arr) & (arr < b)，但在某些需要动态处理多个条件的循环中，reduce 会很有用。

In [15]:
# 检查 arr 是否都在 (0, 10) 范围内
arr = np.array([1, 5, 8])
conditions = [arr > 0, arr < 10]
# 将多个条件合并：必须同时满足 >0 且 <10
in_range = np.logical_and.reduce(conditions)
print(in_range)  # [True True True]

[ True  True  True]


二维数组：检查“行”或“列”
这是它最实用的地方，配合 axis 参数可以检查矩阵的特定维度。

In [17]:
# 创建一个二维数组
# 假设每一行代表一个学生的各科成绩是否及格 (True=及格, False=挂科)
scores = np.array([[True,  True,  True],   # 学生A 全过
                   [True,  False, True],   # 学生B 挂了一科
                   [True,  True,  False]]) # 学生C 挂了一科

all_passed = np.logical_and.reduce(scores, axis=1)
print(all_passed)  # [ True False False]
# 说明：只有第0个学生（A）全科及格

no_one_failed = np.logical_and.reduce(scores, axis=0)
print(no_one_failed)  # [ True False False]
# 说明：只有第0列（第一科）没有人挂科

[ True False False]
[ True False False]


accumulate：记录足迹
它和 reduce 很像，但它保留中间过程的每一步结果。

np.add.accumulate = np.cumsum (累积和)
np.multiply.accumulate = np.cumprod (累积积)
这在时间序列分析（如股票价格走势）中是核心操作。

In [19]:
# 假设初始股价是 100元，后面是每天的变化率（百分比）
initial_price = 100
daily_returns = np.array([1.02, 0.98, 1.05, 0.92, 1.10])
# 2%涨, 2%跌, 5%涨, 8%跌, 10%涨

# 计算累积收益率 (这就是 np.cumprod)
cumulative_returns = np.multiply.accumulate(daily_returns)
print("累积收益率:", cumulative_returns)
# 输出: [1.02 0.9996 1.04958 0.9656124 1.06217364]

# 计算每天的实际股价
stock_price = initial_price * cumulative_returns
print("每日股价:", stock_price)
# 输出: [102. 99.96 104.958 ...] -> 这就是一条完整的K线轨迹！

累积收益率: [1.02       0.9996     1.04958    0.9656136  1.06217496]
每日股价: [102.        99.96     104.958     96.56136  106.217496]


In [20]:
arr_2d = np.array([[1, 2, 3],
                   [4, 5, 6]])

# axis=0: 沿着行记录（垂直方向累加）
print(np.add.accumulate(arr_2d, axis=0))
# 输出:
# [[1 2 3]       <- 第一行原样
#  [5 7 9]]      <- 第一行 + 第二行

# axis=1: 沿着列记录（水平方向累加）
print(np.add.accumulate(arr_2d, axis=1))
# 输出:
# [[ 1  3  6]    <- 第一行内部累加: 1, 1+2, 1+2+3
#  [ 4  9 15]]   <- 第二行内部累加: 4, 4+5, 4+5+6

[[1 2 3]
 [5 7 9]]
[[ 1  3  6]
 [ 4  9 15]]


np.multiply.outer: 这是外积操作。它会把第一个数组的每个元素和第二个数组的每个元素相乘。

在 NumPy 中，np.outer 的作用是计算两个向量的外积。你可以把它理解为：拿第一个向量的每一个元素，去乘以第二个向量的全部元素，从而铺开一张二维的网12。


In [23]:
# 一行代码搞定乘法表
table = np.multiply.outer(np.arange(10), np.arange(10))
print(table)

[[ 0  0  0  0  0  0  0  0  0  0]
 [ 0  1  2  3  4  5  6  7  8  9]
 [ 0  2  4  6  8 10 12 14 16 18]
 [ 0  3  6  9 12 15 18 21 24 27]
 [ 0  4  8 12 16 20 24 28 32 36]
 [ 0  5 10 15 20 25 30 35 40 45]
 [ 0  6 12 18 24 30 36 42 48 54]
 [ 0  7 14 21 28 35 42 49 56 63]
 [ 0  8 16 24 32 40 48 56 64 72]
 [ 0  9 18 27 36 45 54 63 72 81]]
